In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

sys.path.append('../')

from models.csrnet_pytorch.src import *
from models.csrnet_pytorch.src.crowd_count import *
from models.csrnet_pytorch.src.network import *
from models.csrnet_pytorch.src.data_loader import ImageDataLoader
from models.csrnet_pytorch.src.timer import *
from models.csrnet_pytorch.src.evaluate_model import *
from models.csrnet_pytorch.src import utils

In [2]:
# Check to see if device can be trained on GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)


cuda


In [3]:
# Cuda configurations

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

current_device = torch.cuda.current_device()
current_device_name = torch.cuda.get_device_name(current_device)
memory_allocated = torch.cuda.memory_allocated()
memory_cached = torch.cuda.memory_reserved()

print(
    f'Using gpu {current_device_name} with device number {current_device}.\n'
    f'Memory allocated = {memory_allocated}\n'
    f'Memory cached = {memory_cached}'
)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [4]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [5]:
# Directory Configurations

method = 'csrnet'
dataset_name = 'JHU'
output_dir = f'../output/{method}/saved_models/{dataset_name}'

# Training data path
train_path = '../data/JHU/train/consolidated'
train_gt_path = '../data/JHU/train/gt'

# Validation data path
val_path = '../data/JHU/val/consolidated'
val_gt_path = '../data/JHU/val/gt'

In [6]:
# Create output directory if it doesnt exist

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [7]:
# load model

is_cuda = True  # Determine if we should use the CPU to train or GPU

model = CrowdCounter(is_cuda=is_cuda)  # is_cuda determines if all the input tensors should be converted to cuda tensors
network.weights_normal_init(model, dev=0.01)
model.train()

CrowdCounter(
  (model): CSRNet(
    (column): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (1): Conv2d(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (4): Conv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (7): Conv2d(
        (conv): Conv2d(256

In [8]:
# Model parameters

for name, param in model.named_parameters():
    print(f'{name}\t{param.device}\t{param.shape}')

# Print model's state_dict
print("\nModel's state_dict: \n")
for k, v in model.state_dict().items():
    print(k, "\t", v.dtype)

model.column.0.conv.weight	cpu	torch.Size([64, 3, 3, 3])
model.column.0.conv.bias	cpu	torch.Size([64])
model.column.1.conv.weight	cpu	torch.Size([64, 64, 3, 3])
model.column.1.conv.bias	cpu	torch.Size([64])
model.column.3.conv.weight	cpu	torch.Size([128, 64, 3, 3])
model.column.3.conv.bias	cpu	torch.Size([128])
model.column.4.conv.weight	cpu	torch.Size([128, 128, 3, 3])
model.column.4.conv.bias	cpu	torch.Size([128])
model.column.6.conv.weight	cpu	torch.Size([256, 128, 3, 3])
model.column.6.conv.bias	cpu	torch.Size([256])
model.column.7.conv.weight	cpu	torch.Size([256, 256, 3, 3])
model.column.7.conv.bias	cpu	torch.Size([256])
model.column.8.conv.weight	cpu	torch.Size([256, 256, 3, 3])
model.column.8.conv.bias	cpu	torch.Size([256])
model.column.10.conv.weight	cpu	torch.Size([512, 256, 3, 3])
model.column.10.conv.bias	cpu	torch.Size([512])
model.column.11.conv.weight	cpu	torch.Size([512, 512, 3, 3])
model.column.11.conv.bias	cpu	torch.Size([512])
model.column.12.conv.weight	cpu	torch.Siz

In [9]:
# Change model weights tensors to be cuda tensors if is_cuda is true and cuda is available

if is_cuda and torch.cuda.is_available():
    print("Changing to cuda weights")
    model.cuda()

Changing to cuda weights


In [10]:
#training configuration

momentum = 0.9
disp_interval = 1
log_interval = 250

train_loss = 0
step_cnt = 0
re_cnt = False
t = Timer()
t.tic()

# Set initial values
best_mae, best_mse, best_epoch = 999999, 999999, 0

In [11]:
# Hyperparameters

learning_rate = 0.0001
epochs = 3

# construct an optimizer

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=learning_rate)

In [12]:
# Load the images, take note the num_pool argument

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False, num_pool = 3)
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False, num_pool = 3)

print('Training instances: {}'.format(data_loader.get_num_samples()))
print('Validation instances: {}'.format(data_loader_val.get_num_samples()))

Training instances: 500
Validation instances: 100


In [13]:
# Dataset Analysis

def AnalyseLoader(loader, title='Breakdown of the images in the loader by type'):
    d = {'High': 0, 'Med': 0, 'Low': 0, 'None': 0, 'Fog': 0, 'Rain': 0, 'Snow': 0}
    colours = ['orange', 'orange', 'orange', 'green', 'green', 'green', 'green']
    labels = ['crowd', 'weather', 'weather', 'weather', 'weather', 'weather', 'weather']
    for id, blob in enumerate(loader):
        metadata = blob['metadata']
        d[metadata['crowd_density']] += 1
        d[metadata['weather']] += 1
    for i, key in enumerate(d):
        plt.text(i, d[key], '{}%'.format(d[key]*100/loader.get_num_samples()), 
                 ha='center', va='bottom', size=10)
    pd.Series(d).plot(kind='bar', figsize=(8,5), color=colours)
    leg1 = mpatches.Patch(color='orange', label='Crowd Density')
    leg2 = mpatches.Patch(color='green', label='Weather')
    plt.legend(handles=[leg1, leg2])
    plt.title(title)
    plt.show()
    
#AnalyseLoader(data_loader, title='train')
#AnalyseLoader(data_loader_val, title='val')

In [14]:
#Tensorboard  config
use_tensorboard = True

writer = SummaryWriter(f'../output/tensorboard/runs/{learning_rate}_{epochs}')

In [15]:
for epoch in range(epochs):    
    train_loss = 0.0
    for id, blob in enumerate(data_loader, 1):  
        im_data = blob['data']
        gt_data = blob['gt_density']
        metadata = blob['metadata']
                        
        # Set the parameter gradients to zero
        optimizer.zero_grad()

        # Forward pass + backward pass + optimise
        try:
            density_map = model(im_data, gt_data)
        except:
            continue
        loss = model.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        
        # Write to tensorboard
        train_loss += loss.item()
        writer.add_scalar("Loss/train", train_loss, epoch)

        if id % disp_interval == 0:            
            duration = t.toc(average=False)
            fps = step_cnt / duration
            gt_count = np.sum(gt_data)    

            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            utils.save_results(im_data,gt_data,density_map, output_dir)
            #log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %7.1f, et_cnt: %7.1f' % (epoch,step, 1./1, gt_count, et_count)
            log_text = '[%3d, %3d] Actual Count: %6d, Estimated Count: %6.1f' % (epoch+1, id, gt_count, et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True
       
        if re_cnt:                                
            t.tic()
            re_cnt = False
            
        # if id+1 == 10: break
    
    # Overwrite the current model weights
    current_model = f'{method}_{learning_rate}.h5'
    save_name = os.path.join(output_dir, current_model)
    network.save_net(save_name, model)
            
    # Evaluate the mae and mse results by doing a forward pass against the validation dataset i.e data_loader_val for
    # each epoch
    MAEcrowddensity, MSEcrowddensity, MAEweather, MSEweather, MAE, MSE, RMSE = evaluate_model(save_name, data_loader_val, is_cuda=is_cuda)
    
    # Pocket algorithm: Check to see if the current epoch mae is better than the best recorded one,
    # If it is, then overwrite the current best .h5 weights file
    if MAE < best_mae:
        # Save the new best mae and mse
        best_mae = MAE
        best_mse = MSE
        best_epoch = epoch
        best_model = f'best_model_epoch_{best_epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        network.save_net(save_name, model)
        
    # Print out the best epoch that beat the current best mae
    log_text = f'EPOCH: {epoch+1}, Val MAE: {MAE}, Val MSE: {MSE}'
    log_print(log_text, color='blue', attrs=['bold'])

    # Save the results to tensorboard for each epoch
    if use_tensorboard:
        
        # overall segment
        writer.add_scalar("Overall/Val MAE", MAE, epoch)
        writer.add_scalar("Overall/Val MSE", MSE, epoch)
        writer.add_scalar("Overall/Val RMSE", RMSE, epoch)
        writer.add_scalar("Overall/Train MSE", train_loss / data_loader.get_num_samples(), epoch)
        writer.add_scalar("Overall/Train RMSE", np.sqrt(train_loss / data_loader.get_num_samples()), epoch)
        
        # crowd density segment
        writer.add_scalar('Crowd Density/High/MAE', MAEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/MSE', MSEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/RMSE', np.sqrt(MSEcrowddensity['High']), epoch)
        
        writer.add_scalar('Crowd Density/Med/MAE', MAEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/MSE', MSEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/RMSE', np.sqrt(MSEcrowddensity['Med']), epoch)
        
        writer.add_scalar('Crowd Density/Low/MAE', MAEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/MSE', MSEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/RMSE', np.sqrt(MSEcrowddensity['Low']), epoch)
        
        # weather segment
        writer.add_scalar('Weather/No Degradation/MAE', MAEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/MSE', MSEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/RMSE', np.sqrt(MSEweather['None']), epoch)
        
        writer.add_scalar('Weather/Fog/MAE', MAEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/MSE', MSEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/RMSE', np.sqrt(MSEweather['Fog']), epoch)
        
        writer.add_scalar('Weather/Rain/MAE', MAEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/MSE', MSEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/RMSE', np.sqrt(MSEweather['Rain']), epoch)
        
        writer.add_scalar('Weather/Snow/MAE', MAEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/MSE', MSEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/RMSE', np.sqrt(MSEweather['Snow']), epoch)

[  1,   1] Actual Count:     40, Estimated Count:    0.0
[  1,   2] Actual Count:     76, Estimated Count:   26.5
[  1,   3] Actual Count:    756, Estimated Count:  512.9
[  1,   4] Actual Count:    839, Estimated Count: 1447.8


KeyboardInterrupt: 